In [4]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.model_selection import train_test_split


In [2]:
train_set = pd.read_csv('/kaggle/input/svmdataset/trainClean.csv')
test_set = pd.read_csv('/kaggle/input/svmdataset/testClean.csv')
val_set = pd.read_csv('/kaggle/input/svmdataset/valClean.csv')

In [3]:
X_train = train_set.iloc[:, :-1]
y_train = train_set.iloc[:, -1]
X_test = test_set.iloc[:, :-1]
y_test = test_set.iloc[:, -1]
X_val = val_set.iloc[:, :-1]
y_val = val_set.iloc[:, -1]

In [4]:
num_negative_class = sum(y_train == 0)
num_positive_class = sum(y_train == 1)

class_ratio = num_negative_class / num_positive_class

print(f'Class Ratio: {class_ratio}')

Class Ratio: 1.7253914988814318


/tmp/ipykernel_26/4187596743.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misclassified_instances[X_test.columns] = X_test.loc[misclassified_instances.index]
/tmp/ipykernel_26/4187596743.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  misclassified_instances[X_test.columns] = X_test.loc[misclassified_instances.index]
/tmp/ipykernel_26/4187596743.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [6]:
misclassified_instances


,True_Label,Predicted_Label,Correctly_Classified,age,gender_F,BMI,waist(cm),Blood Pressure,fasting blood sugar,Cholesterol,triglyceride,LDL/HDL ratio,hemoglobin,serum creatinine,AST/ALT ratio,Gtp
10,0,1,False,60,0,29.296875,96.0,2,216.0,260.0,335.0,2.784314,15.6,0.7,0.736842,56.0
14,1,0,False,75,0,22.038567,88.0,1,103.0,209.0,176.0,2.866667,14.7,1.0,1.866667,116.0
17,0,1,False,55,0,31.020408,92.0,1,131.0,192.0,328.0,1.739130,16.3,1.0,0.974359,72.0
27,0,1,False,40,0,24.691358,87.0,2,125.0,190.0,251.0,3.088235,15.2,0.8,0.560000,44.0
32,1,0,False,30,0,22.038567,85.0,2,98.0,205.0,68.0,1.437500,14.9,0.9,1.157895,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8339,0,1,False,30,0,26.234568,98.0,2,88.0,198.0,149.0,1.270270,15.6,1.0,0.750000,57.0
8341,0,1,False,25,0,20.761246,79.0,2,88.0,170.0,180.0,2.367347,15.8,0.9,1.058824,26.0
8343,0,1,False,45,0,24.221453,80.0,2,106.0,183.0,126.0,2.434783,14.4,1.2,1.260870,31.0
8350,1,0,False,30,0,24.489796,86.0,2,97.0,185.0,90.0,2.479167,15.3,0.9,0.593750,18.0


In [8]:
from IPython.display import FileLink

# Save the DataFrame to a CSV file
misclassified_instances.to_csv('/kaggle/working/misclassified_instances.csv', index=False)

# Create a download link for the file
FileLink(r'misclassified_instances.csv')

/kaggle/working/misclassified_instances.csv

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from imblearn.over_sampling import SMOTE
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, accuracy_score, confusion_matrix
import pandas as pd


train_set = pd.read_csv('/kaggle/input/svmdataset/trainClean.csv')
test_set = pd.read_csv('/kaggle/input/svmdataset/testClean.csv')
val_set = pd.read_csv('/kaggle/input/svmdataset/valClean.csv')

X_train = train_set.iloc[:, :-1]
y_train = train_set.iloc[:, -1]
X_test = test_set.iloc[:, :-1]
y_test = test_set.iloc[:, -1]
X_val = val_set.iloc[:, :-1]
y_val = val_set.iloc[:, -1]
# Define the RandomForestClassifier
rf_classifier = RandomForestClassifier(random_state=42)

# Define the parameter grid to search
param_grid = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
}

# Use SMOTE for handling imbalanced data
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Create a custom scoring metric (e.g., F1-score) for cross-validation
scoring = {'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1_score': make_scorer(f1_score)}

# Create the StratifiedKFold cross-validator
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Create the GridSearchCV object
grid_search = GridSearchCV(estimator=rf_classifier, param_grid=param_grid, scoring=scoring,
                           cv=cv, refit='f1_score', verbose=2, n_jobs=-1)

# Fit the model using the resampled training set
grid_search.fit(X_train_resampled, y_train_resampled)

# Get the best hyperparameters
best_params = grid_search.best_params_

# Evaluate the best model on the validation set
best_model = grid_search.best_estimator_
y_val_pred = best_model.predict(X_val)

# Evaluate performance metrics on the validation set
precision = precision_score(y_val, y_val_pred)
recall = recall_score(y_val, y_val_pred)
f1 = f1_score(y_val, y_val_pred)

# Print results on the validation set
print("Best Hyperparameters:", best_params)
print("Precision on Validation Set:", precision)
print("Recall on Validation Set:", recall)
print("F1 Score on Validation Set:", f1)

# Calculate and print accuracy on the validation set
accuracy_val = accuracy_score(y_val, y_val_pred)
print("Accuracy on Validation Set:", accuracy_val)

# Print confusion matrix on the validation set
conf_matrix_val = confusion_matrix(y_val, y_val_pred)
print("\nConfusion Matrix on Validation Set:")
print(conf_matrix_val)

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)

# Evaluate performance metrics on the test set
precision_test = precision_score(y_test, y_test_pred)
recall_test = recall_score(y_test, y_test_pred)
f1_test = f1_score(y_test, y_test_pred)

# Print results on the test set
print("\nResults on Test Set:")
print("Precision on Test Set:", precision_test)
print("Recall on Test Set:", recall_test)
print("F1 Score on Test Set:", f1_test)

# Calculate and print accuracy on the test set
accuracy_test = accuracy_score(y_test, y_test_pred)
print("Accuracy on Test Set:", accuracy_test)

# Print confusion matrix on the test set
conf_matrix_test = confusion_matrix(y_test, y_test_pred)
print("\nConfusion Matrix on Test Set:")
print(conf_matrix_test)

Fitting 5 folds for each of 256 candidates, totalling 1280 fits


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   6.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   6.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=  13.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=150; total time=  19.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=  26.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   6.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   6.4s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=50; total time=   6.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=  12.6s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=5, n_estimators

In [5]:
# Accuracy, Confusion Matrix, Classification Report for validation set
print("\nValidation Set:")
print("Accuracy:", accuracy_score(y_val, y_val_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_val_pred))
print("Classification Report:\n", classification_report(y_val, y_val_pred))

# Accuracy, Confusion Matrix, Classification Report for test set
print("\nTest Set:")
print("Accuracy:", accuracy_score(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))


Validation Set:
Accuracy: 0.8152980608091932
Confusion Matrix:
 [[4222 1061]
 [ 482 2589]]
Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.80      0.85      5283
           1       0.71      0.84      0.77      3071

    accuracy                           0.82      8354
   macro avg       0.80      0.82      0.81      8354
weighted avg       0.83      0.82      0.82      8354


Test Set:
Accuracy: 0.8107493416327508
Confusion Matrix:
 [[4203 1071]
 [ 510 2570]]
Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.80      0.84      5274
           1       0.71      0.83      0.76      3080

    accuracy                           0.81      8354
   macro avg       0.80      0.82      0.80      8354
weighted avg       0.82      0.81      0.81      8354



In [7]:
from IPython.display import FileLink

# Evaluate the best model on the test set
y_test_pred = best_model.predict(X_test)

# Collect information about misclassified instances
misclassified_data = pd.DataFrame({
    'True_Label': y_test,
    'Predicted_Label': y_test_pred,
    'Correctly_Classified': y_test == y_test_pred  # True if correctly classified, False if misclassified
})

# Filter misclassified instances
error_instances = misclassified_data[misclassified_data['Correctly_Classified'] == False]

# Add input features to the DataFrame
error_instances[X_test.columns] = X_test.loc[error_instances.index]

error_instances.to_csv('/kaggle/working/error_instances.csv', index=False)

# Create a download link for the file
FileLink(r'error_instances.csv')

/tmp/ipykernel_26/2333341733.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_instances[X_test.columns] = X_test.loc[error_instances.index]
/tmp/ipykernel_26/2333341733.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  error_instances[X_test.columns] = X_test.loc[error_instances.index]
/tmp/ipykernel_26/2333341733.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

/kaggle/working/error_instances.csv